In [2]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from gensim.models.doc2vec import Doc2Vec,TaggedDocument 
import nltk
from nltk.corpus import stopwords
import os
import seaborn as sns
from nltk.tokenize import casual_tokenize

In [3]:
import multiprocessing

In [35]:
num_cores=multiprocessing.cpu_count()

In [4]:
path=r"C:\Users\iavta\Natural Language Processing\Economics\FED\\"
df_Fed_merged=pd.read_csv(path+"Data - Clean\\"+"Fed_merged.csv")

In [5]:
df_Fed_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855 entries, 0 to 854
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 855 non-null    object 
 1   link                 855 non-null    object 
 2   title                855 non-null    object 
 3   event                855 non-null    object 
 4   text                 855 non-null    object 
 5   location             855 non-null    object 
 6   DATE                 855 non-null    object 
 7   T5YIFR               855 non-null    float64
 8   Changes of T5YIFR    855 non-null    float64
 9   Distance to 2        855 non-null    float64
 10  Changes in Distance  855 non-null    float64
 11  Classes              855 non-null    float64
 12  T5YIFR Lagged        855 non-null    float64
dtypes: float64(6), object(7)
memory usage: 87.0+ KB


In [6]:
data_X=df_Fed_merged[["text","T5YIFR Lagged"]]

In [7]:
def Time_Validation(datafr,labels,test_size=0.2):
    train_index = list(range(int(len(datafr)-np.floor(test_size*len(datafr))+1)))
    test_index =list(range(int(len(datafr)-np.floor(test_size*len(datafr))+1),len(datafr)))
    return datafr.loc[train_index], datafr.loc[test_index],labels[train_index], labels[test_index]

In [18]:
def prepare_corpus(array, tokens_only=False):
    for i, text in enumerate(array):
        tokens = casual_tokenize(text)
        if tokens_only:
            yield tokens
        else:
            # For training data (only), add tags
            yield TaggedDocument(tokens, [i])

In [29]:
def prepare_corpus_comp(array, tokens_only=False):
    [casual_tokenize(text) if tokens_only 
     else TaggedDocument(casual_tokenize(text), [i]) for i, text in enumerate(array)]

# Fed - LGBM - Classification

In [8]:
from lightgbm import LGBMClassifier

In [9]:
labels=np.asarray(df_Fed_merged["Classes"].astype("int"))
labels.shape

(855,)

In [10]:
data_X_train, data_X_test, labels_train, labels_test=Time_Validation(data_X,labels,test_size=0.05)

In [64]:
train_size , _= data_X_train.shape
test_size , _= data_X_test.shape

In [28]:
%timeit corpus_train=list(prepare_corpus(data_X_train["text"])) 
# yield performs very similarly to list comprehension on avg, but with less variation.

34.4 s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit corpus_train=prepare_corpus_comp(data_X_train["text"])

34.6 s ± 2.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
corpus_train=list(prepare_corpus(data_X_train["text"]))

#instantiating Doc2Vec class
vector_size=50
embedding_doc2vec=Doc2Vec(vector_size=vector_size, min_count=2, epochs=40, workers=num_cores)

#building and training model
embedding_doc2vec.build_vocab(corpus_train)

embedding_doc2vec.train(corpus_train,total_examples=embedding_doc2vec.corpus_count,
                       epochs=embedding_doc2vec.epochs)

In [113]:
path_saved_LGBM=path+"Saved Models\\"+"Embedding-Doc2Vec\\"
# we need to create the directory before running the command below
embedding_doc2vec.save(path_saved_LGBM+"embedding_doc2vec_classifier")
# to load, uncomment the line below
# embedding_doc2vec = Doc2Vec.load(path_saved_LGBM+"embedding_doc2vec")

In [74]:
# embedding_doc2vec.dv is gensim keyedvector data type,
# with each component beint the document vector (dv) obtained from training

df_train=pd.DataFrame(np.array([embedding_doc2vec.dv[i] for i in range(train_size)]),
                      columns=["doc"+str(i) for i in range(vector_size)])

df_train["T5YIFR Lagged"]=data_X_train["T5YIFR Lagged"]

In [75]:
df_train.head()

,doc0,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,...,doc41,doc42,doc43,doc44,doc45,doc46,doc47,doc48,doc49,T5YIFR Lagged
0,-2.488771,-2.538723,-2.873959,-3.112880,1.295459,-5.347455,-1.853269,1.875832,-2.946009,-1.938024,...,1.638317,1.559110,-0.896035,1.508760,-1.582389,-0.960373,-0.457239,0.185955,-0.440521,2.13
1,-2.726032,-2.394056,0.864733,-3.695441,-0.792782,-1.818849,-0.164185,3.601535,-3.283093,1.250679,...,0.699519,-0.904267,-1.933889,0.610341,-3.263781,0.654586,-0.718167,0.417214,-0.467186,2.14
2,-0.687704,-4.629267,0.523481,-0.605821,0.672305,-0.483044,-1.052666,1.835982,-1.098219,-0.777684,...,-1.029462,-0.724765,-0.600786,-0.079076,-1.886830,0.452371,-2.356006,0.255244,-0.792763,2.19
3,-1.668830,-4.357888,0.393071,-1.464142,-2.265313,-0.343736,1.392451,0.493134,-2.076980,-1.469864,...,-2.001611,1.527520,-1.647809,-1.148558,-1.388819,-0.790706,-3.317306,3.524722,-1.668324,2.20
4,-1.172548,-2.807615,-3.059715,-2.474043,0.305137,-5.513704,-2.642413,2.272916,-2.565372,-0.554859,...,2.796026,1.482919,0.309122,-0.407435,-1.756124,-0.040547,0.077617,-0.188567,-0.838324,2.24


In [76]:
lgbmc=LGBMClassifier();
lgbmc.fit(df_train,labels_train)

LGBMClassifier()

In [79]:
corpus_test=list(prepare_corpus(data_X_test["text"],tokens_only=True))

df_test=pd.DataFrame(np.array([embedding_doc2vec.infer_vector(corpus_test[i])
                                for i in range(test_size)]),
                      columns=["doc_test"+str(i) for i in range(vector_size)])
df_test["T5YIFR Lagged"]=data_X_test["T5YIFR Lagged"]

In [81]:
predictions=lgbmc.predict(df_test)
probabilities=lgbmc.predict_proba(df_test)

In [82]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.metrics import roc_auc_score,classification_report

In [83]:
confusion_matrix(labels_test, predictions)
#  i-th row and j-th column entry indicates the number of samples 
#  with true label being i-th class and predicted label being j-th class.

array([[1, 2, 4],
       [2, 7, 9],
       [0, 9, 7]], dtype=int64)

We can see that lgbm still tends to predict class 2 (from 0,1 and 2) too many times, but less than if we hadn't used the doc2vec embedding.

In [85]:
balanced_accuracy_score(labels_test, predictions,adjusted=True) 
# accuracy score for imbalanced classes
# 0 for a random classifier
# 1 for a perfect classifier

-0.015376984126984098

In [86]:
roc_auc_score(labels_test,probabilities, multi_class="ovr")

0.5255554202357271

This value for roc seems to indicate performance is worse than a random classifier which would get a roc of 0.5...

I think I need to do a random grid search to the parameters of the LGBM classifier. 

In [115]:
examples=[
    'Trust us, inflation will be 2%',
    'We are unsure whether it will be 2%',
    'Inflation will be higher than 2%',
    'Inflation will be lower than 2%',
]

examples_trans = list(prepare_corpus(examples,tokens_only=True))
examples_trans=pd.DataFrame(np.array([embedding_doc2vec.infer_vector(i)
                                for i in examples_trans]))

examples_trans["T5YIFR Lagged"]=[3,3,3,3] 
# I've tried several different vectors

In [119]:
predictions = lgbmc.predict(examples_trans)
# 0 ->  no change
# 2 -> increased the distance relative to objective of 2
# 1 -> decreased the distance relative to objective of 2


for sent,pred in zip(examples,predictions):
    print("Sentence: {0} Predicted Class: {1} \n".format(sent,pred))

Sentence: Trust us, inflation will be 2% Predicted Class: 2 

Sentence: We are unsure whether it will be 2% Predicted Class: 2 

Sentence: Inflation will be higher than 2% Predicted Class: 0 

Sentence: Inflation will be lower than 2% Predicted Class: 0 



# Fed - LGBM - Regression

In [120]:
from lightgbm import LGBMRegressor

In [121]:
target=np.asarray(df_Fed_merged["T5YIFR"])
target.shape

(855,)

In [122]:
data_X_train, data_X_test, target_train, target_test=Time_Validation(data_X,target,test_size=0.05)

In [124]:
train_size , _= data_X_train.shape
test_size , _= data_X_test.shape

corpus_train=list(prepare_corpus(data_X_train["text"]))

#instantiating Doc2Vec class
vector_size=50
embedding_doc2vec=Doc2Vec(vector_size=vector_size, min_count=2, epochs=40, workers=num_cores)

#building and training model
embedding_doc2vec.build_vocab(corpus_train)

embedding_doc2vec.train(corpus_train,total_examples=embedding_doc2vec.corpus_count,
                       epochs=embedding_doc2vec.epochs)

In [125]:
path_saved_LGBM=path+"Saved Models\\"+"Embedding-Doc2Vec\\"
# we need to create the directory before running the command below
embedding_doc2vec.save(path_saved_LGBM+"embedding_doc2vec_regressor")
# to load, uncomment the line below
# embedding_doc2vec = Doc2Vec.load(path_saved_LGBM+"embedding_doc2vec")

In [126]:
# embedding_doc2vec.dv is gensim keyedvector data type,
# with each component beint the document vector (dv) obtained from training

df_train=pd.DataFrame(np.array([embedding_doc2vec.dv[i] for i in range(train_size)]),
                      columns=["doc"+str(i) for i in range(vector_size)])

df_train["T5YIFR Lagged"]=data_X_train["T5YIFR Lagged"]

In [129]:
lgbm_reg=LGBMRegressor()
lgbm_reg.fit(df_train,target_train)

LGBMRegressor()

In [130]:
corpus_test=list(prepare_corpus(data_X_test["text"],tokens_only=True))

df_test=pd.DataFrame(np.array([embedding_doc2vec.infer_vector(corpus_test[i])
                                for i in range(test_size)]),
                      columns=["doc_test"+str(i) for i in range(vector_size)])
df_test["T5YIFR Lagged"]=data_X_test["T5YIFR Lagged"]

In [133]:
predictions_reg=lgbm_reg.predict(df_test)

In [134]:
accuracy_reg = lgbm_reg.score(df_test,target_test)
print(accuracy_reg) #

rmse = np.sqrt(np.mean((predictions_reg-target_test)**2))
print(rmse)

-4.104107956812326
0.22814947914167963


In [135]:
examples_trans = list(prepare_corpus(examples,tokens_only=True))
examples_trans=pd.DataFrame(np.array([embedding_doc2vec.infer_vector(i)
                                for i in examples_trans]))

examples_trans["T5YIFR Lagged"]=[3,3,3,3] 
# I've tried several different vectors

predictions = lgbm_reg.predict(examples_trans)
# 0 ->  no change
# 2 -> increased the distance relative to objective of 2
# 1 -> decreased the distance relative to objective of 2
for sent,pred in zip(examples,predictions):
    print("Sentence: {0} Predicted Class: {1} \n".format(sent,pred))

Sentence: Trust us, inflation will be 2% Predicted Class: 2.849320202723464 

Sentence: We are unsure whether it will be 2% Predicted Class: 2.881743185562295 

Sentence: Inflation will be higher than 2% Predicted Class: 2.8730307640072703 

Sentence: Inflation will be lower than 2% Predicted Class: 2.873259804161956 

